In [ ]:
import pandas as pd
import io
import requests

def ler_planilha_corrigir_datas(caminho_arquivo):
    """
    Função para ler a planilha e corrigir a coluna de datas para o formato adequado.

    Parâmetros:
    - caminho_arquivo: Caminho do arquivo CSV.

    Retorno:
    - DataFrame pandas com a coluna de datas convertida corretamente.
    """
    # Carregar o arquivo CSV usando o delimitador correto ";"
    df = pd.read_csv(caminho_arquivo, delimiter=';')

    # Dicionário para traduzir os meses de português para inglês
    meses_portugues = {
        'janeiro': 'January', 'fevereiro': 'February', 'março': 'March', 'abril': 'April',
        'maio': 'May', 'junho': 'June', 'julho': 'July', 'agosto': 'August',
        'setembro': 'September', 'outubro': 'October', 'novembro': 'November', 'dezembro': 'December'
    }

    # Substituir os meses em português por seus equivalentes em inglês na coluna de datas
    for mes_pt, mes_en in meses_portugues.items():
        df['Semana de Data'] = df['Semana de Data'].str.replace(mes_pt, mes_en)

    # Remover "de" da string de data (ex: "27 de December de 2004" -> "27 December 2004")
    df['Semana de Data'] = df['Semana de Data'].str.replace(r'\s+de\s+', ' ', regex=True)

    # Converter a coluna de datas para o formato datetime (formato: "%Y %B %d")
    df['Semana de Data'] = pd.to_datetime(df['Semana de Data'], format='%d %B %Y')

    return df

def selecionar_precos_energia(df_corrigido, ano_inicio, periodo):
    """
    Seleciona os preços de energia para o período desejado com base no ano de início e no período (meses).

    Parâmetros:
    - df_corrigido: DataFrame da planilha de preços de energia corrigido.
    - ano_inicio: Ano de início da simulação.
    - periodo: Número de meses a serem simulados.

    Retorno:
    - Lista de preços de energia mensais para o período selecionado.
    """
    # Filtrar os dados a partir do ano de início
    df_filtrado = df_corrigido[df_corrigido['Semana de Data'].dt.year >= ano_inicio]

    # Garantir que os dados estão ordenados por data
    df_filtrado = df_filtrado.sort_values(by='Semana de Data')

    # Selecionar o número de meses correspondentes ao período
    df_selecionado = df_filtrado.head(periodo)

    # Retornar a lista de preços de energia (coluna 'Média Mensal (R$/MWh)')
    return (df_selecionado['Média Mensal (R$/MWh)'].tolist())

import numpy as np

#add referencia com o DOI

class Eletrolisador:
    def __init__(self, eficiencia_potencia, tensao_corrente, proporcao_h2_max_eficiencia, agua_hidrogenio, tipo="Alcalino"):
        """
        Inicializa os dados do eletrolisador.

        :param eficiencia_potencia: Lista de pares (potencia, eficiencia) da curva de eficiencia.
        :param tensao_corrente: Lista de pares (corrente, tensão) da relação tensão-corrente.
        :param proporcao_h2_max_eficiencia: Proporção de H2 produzido por kWh no ponto de máxima eficiência (kg/kWh).
        :param agua_hidrogenio: Par ordenado (massa de H2, água consumida no ponto de máxima eficiência).
        :param tipo: Tipo do eletrolisador ("Alcalino" ou "PEM").
        """
        self.eficiencia_potencia = eficiencia_potencia  # [(P1, eta1), (P2, eta2), ...]
        self.tensao_corrente = tensao_corrente          # [(I1, V1), (I2, V2), ...]
        self.proporcao_h2_max_eficiencia = proporcao_h2_max_eficiencia  # kg de H2 por kWh no ponto de máxima eficiência
        self.agua_hidrogenio = agua_hidrogenio          # (massa de H2, água consumida) no ponto de máxima eficiência

        # Verifica se o tipo é válido
        if tipo not in ["Alcalino", "PEM"]:
            raise ValueError("Tipo de eletrolisador inválido. Escolha 'Alcalino' ou 'PEM'.")

        self.tipo = tipo  # Define o tipo do eletrolisador

    def ajustar_para_potencia_nominal(self, potencia_nominal):
        """
        Ajusta os valores de potência da curva de eficiência com base na nova potência nominal.
        Escala os valores de potência da curva para a potência nominal fornecida pelo usuário.
        """
        potencias_originais, eficiencias = zip(*self.eficiencia_potencia)
        fator_ajuste = potencia_nominal / max(potencias_originais)  # Ajuste proporcional
        potencias_ajustadas = [p * fator_ajuste for p in potencias_originais]
        self.eficiencia_potencia = list(zip(potencias_ajustadas, eficiencias))

    def encontrar_ponto_maxima_eficiencia(self):
        """
        Retorna o ponto de potência de máxima eficiência na curva.
        """
        potencias, eficiencias = zip(*self.eficiencia_potencia)
        # Ajusta um polinômio de grau 4 aos dados
        coef = np.polyfit(potencias, eficiencias, 4)
        polinomio = np.poly1d(coef)
        # Gerar pontos suavizados para encontrar o máximo
        potencias_suavizadas = np.linspace(min(potencias), max(potencias), 1000)
        eficiencias_suavizadas = polinomio(potencias_suavizadas)
        indice_max = np.argmax(eficiencias_suavizadas)
        return potencias_suavizadas[indice_max], eficiencias_suavizadas[indice_max]  # Potência e eficiência máxima

    def eficiencia_em_funcao_da_potencia(self, potencia):
        """
        Retorna a eficiência correspondente para uma dada potência usando um polinômio de grau 4.
        """
        potencias, eficiencias = zip(*self.eficiencia_potencia)
        # Ajusta um polinômio de grau 4 aos dados
        coef = np.polyfit(potencias, eficiencias, 4)
        # Avalia o polinômio na potência desejada
        polinomio = np.poly1d(coef)
        return polinomio(potencia)

    def calcular_producao_hidrogenio(self, horas_por_dia, dias_por_semana, percentual_potencia, potencia_nominal):
        """
        Calcula a produção de hidrogênio (kg), energia consumida (kWh), degradação, consumo de água e proporções.
        """
        # Potência nominal está em kW, então convertemos para W (1 kW = 1000 W)
        potencia_nominal_watts = potencia_nominal * 1000

        # Cálculo da energia consumida (kWh)
        energia_consumida = potencia_nominal_watts * (percentual_potencia / 100) * horas_por_dia * dias_por_semana / 1000

        # Interpolar a eficiência com base na potência (potência real em W)
        potencia_operacao = potencia_nominal_watts * (percentual_potencia / 100)
        eficiencia = self.eficiencia_em_funcao_da_potencia(potencia_operacao)

        # Ajustar a produção de hidrogênio com base na eficiência em relação ao ponto de máxima eficiência
        # No ponto de máxima eficiência, a proporção é fixa: self.proporcao_h2_max_eficiencia (kg/kWh)
        proporcao_h2_operacao = self.proporcao_h2_max_eficiencia * (eficiencia / 1.0)

        # Cálculo da produção de hidrogênio (kg)
        quantidade_hidrogenio = proporcao_h2_operacao * energia_consumida

        # Degradação do eletrolisador (simplificada)
        degradacao = 0.001 * horas_por_dia * dias_por_semana

        # Proporção de água consumida por kg de H2 no ponto de máxima eficiência
        massa_h2_max, agua_consumida_max = self.agua_hidrogenio  # Proporção no ponto de máxima eficiência
        proporcao_agua_h2 = agua_consumida_max / massa_h2_max

        # O consumo de água depende diretamente da quantidade de H2 produzido, independentemente da eficiência
        agua_consumida = proporcao_agua_h2 * quantidade_hidrogenio

        # Cálculo da proporção de água consumida por kWh no ponto de operação
        proporcao_agua_por_kwh = agua_consumida / energia_consumida

        # Cálculo da proporção de hidrogênio produzido por kWh
        proporcao_h2_por_kwh = quantidade_hidrogenio / energia_consumida

        return quantidade_hidrogenio, energia_consumida, degradacao, agua_consumida, proporcao_agua_por_kwh, proporcao_h2_por_kwh


# Função para perguntar ao usuário se ele quer usar os dados padrão ou fornecer os próprios dados
def obter_dados_eletrolisador():
    escolha = input("Você gostaria de usar os dados padrão (s/n)? ").lower()

    if escolha == 's':
        # Dados padrão (potências em W)
        dados_eficiencia = [
            (1000, 0.52),  # Eficiência em 1000 W
            (2000, 0.58),  # Eficiência em 2000 W
            (3000, 0.58),  # Pico de eficiência em 3000 W
            (4000, 0.56),  # Queda gradual após 3000 W
            (5000, 0.55)   # Eficiência em 5000 W
        ]
        dados_tensao = [(100, 1.8), (200, 2.0), (300, 2.2)]
        proporcao_h2_max_eficiencia = 0.05  # Exemplo de 0.05 kg de H2 por kWh no ponto de máxima eficiência
        dados_agua_hidrogenio = (1.0, 9.0)  # Para cada 1kg de H2 produzido, 9kg de água consumidos na eficiência máxima
    else:
        # Coletar os dados do usuário
        dados_eficiencia = []
        dados_tensao = []
        dados_agua_hidrogenio = None

        print("Forneça os dados de eficiência (potência em kW e eficiência correspondente):")
        for _ in range(3):
            potencia = float(input("Potência (kW): ")) * 1000  # Converte kW para W
            eficiencia = float(input("Eficiência (entre 0 e 1): "))
            dados_eficiencia.append((potencia, eficiencia))

        print("\nForneça os dados de tensão (corrente em A e tensão correspondente):")
        for _ in range(3):
            corrente = float(input("Corrente (A): "))
            tensao = float(input("Tensão (V): "))
            dados_tensao.append((corrente, tensao))

        print("\nForneça a proporção de H2 produzido por kWh no ponto de máxima eficiência (kg/kWh):")
        proporcao_h2_max_eficiencia = float(input("Proporção de H2 (kg/kWh): "))

        print("\nForneça os dados de hidrogênio (massa de H2 em kg e água consumida em litros no ponto de máxima eficiência):")
        massa_hidrogenio = float(input("Massa de H2 (kg): "))
        agua_consumida = float(input("Água consumida (litros): "))
        dados_agua_hidrogenio = (massa_hidrogenio, agua_consumida)

    return dados_eficiencia, dados_tensao, proporcao_h2_max_eficiencia, dados_agua_hidrogenio


def calcular_investimento_adaptado(investimento_inicial, taxa_retorno, preco_venda_hidrogenio, df_corrigido, eletrolisador, ano_inicio, periodo):
    # Converter a taxa de retorno para decimal
    taxa_retorno_decimal = ((taxa_retorno / 100) / 12)

    # Cálculo do valor futuro do investimento inicial com juros compostos
    valor_futuro = investimento_inicial * (1 + taxa_retorno_decimal) ** periodo

    # Selecionar os preços de energia para o período desejado
    preco_energia_planilha = selecionar_precos_energia(df_corrigido, ano_inicio, periodo)

    # Cálculo do fluxo de caixa ao longo dos períodos
    fluxo_caixa = []
    saldo = -investimento_inicial  # Começa com o investimento inicial negativo (saída de capital)

  # Solicitar os parâmetros de operação do usuário
    horas_por_dia = float(input("Insira o número de horas de operação por dia: "))
    dias_por_semana = int(input("Insira o número de dias de operação por semana: "))
    percentual_potencia = float(input("Insira o percentual da potência nominal (%): "))
    potencia_nominal = float(input("Insira a potência nominal do eletrolisador (kW): ")) * 1000  # Converte kW para W

  # Ajustar os valores de potência da curva de eficiência para a nova potência nominal
    eletrolisador.ajustar_para_potencia_nominal(potencia_nominal)

    for mes in range(1, periodo + 1):
        # Calcular a produção de hidrogênio e consumo de energia usando a função do eletrolisador
        quantidade_hidrogenio, energia_consumida, degradacao, agua_consumida, proporcao_agua_por_kwh, proporcao_h2_por_kwh = eletrolisador.calcular_producao_hidrogenio(
            horas_por_dia=horas_por_dia,
            dias_por_semana=dias_por_semana,
            percentual_potencia=percentual_potencia,
            potencia_nominal=potencia_nominal/1000  # Converte para kW para o cálculo
        )

        # Cálculo da receita mensal com a venda de hidrogênio
        receita_mensal = quantidade_hidrogenio * preco_venda_hidrogenio
        #print(quantidade_hidrogenio)

        # Cálculo do custo mensal de energia (usando o preço da planilha)
        preco_energia_mes = float(preco_energia_planilha[mes-1].replace(',', '.'))
        #print(preco_energia_mes)

        #print(preco_energia_mes) # Preço da energia para o mês corrente
        custo_mensal = energia_consumida * preco_energia_mes
        #print(energia_consumida)

        # Atualiza o saldo: saldo += receita_mensal - custo_mensal
        saldo += receita_mensal - custo_mensal
        #print(saldo)

        # Armazena o fluxo de caixa mensal com receitas e custos
        fluxo_caixa.append((mes, receita_mensal, custo_mensal, saldo))
        #print(fluxo_caixa)

    # Cálculo do payback (número de meses até o saldo se tornar positivo)
    payback = None
    for mes, receita, custo, saldo in fluxo_caixa:
        if saldo >= 0:
            payback = mes
            break

    # Resultado final
    resultado = {
        "Valor Futuro": valor_futuro,
        "Fluxo de Caixa": fluxo_caixa,
        "Payback (mes)": payback
    }

    print(resultado)
    return resultado

# Definir o caminho do arquivo CSV de energia
caminho = 'https://drive.google.com/uc?export=download&id=18MXSxkCkg1Ck67CGBne5wVT5PfRgAOZZ'
r = requests.get(caminho)
data = io.StringIO(r.text)

# Exemplo de uso da função
df_corrigido = ler_planilha_corrigir_datas(data)


# Solicitando os dados do usuário
dados_eficiencia, dados_tensao, proporcao_h2_max_eficiencia, dados_agua_hidrogenio = obter_dados_eletrolisador()

# Criar o eletrolisador com os dados fornecidos ou padrão
eletrolisador = Eletrolisador(dados_eficiencia, dados_tensao, proporcao_h2_max_eficiencia, dados_agua_hidrogenio, tipo="PEM")



# Executar o cálculo do investimento com os parâmetros fornecidos
resultado = calcular_investimento_adaptado(
    float(input('Insira o investimento inicial:')),  # Exemplo de investimento inicial
    float(input('Insira a taxa de retorno anual:')),              # Exemplo de taxa de retorno anual
    float(input('Insira o preco de venda do hidrogenio:')),   # Preço de venda do hidrogênio por kg
    df_corrigido = df_corrigido,   # DataFrame de preços de energia corrigido
    eletrolisador = eletrolisador,
    ano_inicio= int(input('Insira o ano inicio:')),  #Ano de início da simulação
    periodo= int(input('Insira o periodo:')),# Número de meses a simular
)

Você gostaria de usar os dados padrão (s/n)? s
Insira o investimento inicial:10000
Insira a taxa de retorno anual:5
Insira o preco de venda do hidrogenio:100
Insira o ano inicio:2024
Insira o periodo:3
Insira o número de horas de operação por dia: 6
Insira o número de dias de operação por semana: 5
Insira o percentual da potência nominal (%): 80
Insira a potência nominal do eletrolisador (kW): 500
{'Valor Futuro': 10125.521556712962, 'Fluxo de Caixa': [(1, 33600.00000000001, 733680.0, -710080.0), (2, 33600.00000000001, 734400.0, -1410880.0), (3, 33600.00000000001, 732840.0, -2110120.0)], 'Payback (mes)': None}
